# Дерева рішень. Класифікація. Депозити

Курс: "Математичне моделювання в R"

---

У даній частині навчального процесу потрібно побудувати математичні моделі класифікації клієнтів на основі алгоритму дерева рішень та перевірити їх на тестовій вибірці.

In [ ]:
# install.packages("C50")
# install.packages("xgboost")
#install.packages("scorecard")
#install.packages("lightgbm")
#install.packages("Matrix") 

## Dataset description

**Abstract**

The data is related with direct marketing campaigns (phone calls) of a Portuguese banking institution. The classification goal is to predict if the client will subscribe a term deposit (variable y).



**Data Set Information:**

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

There are four datasets:
1. bank-additional-full.csv with all examples (41188) and 20 inputs, ordered by date (from May 2008 to November 2010), very close to the data analyzed in [Moro et al., 2014]
2. bank-additional.csv with 10% of the examples (4119), randomly selected from 1), and 20 inputs.
3. bank-full.csv with all examples and 17 inputs, ordered by date (older version of this dataset with less inputs).
4. bank.csv with 10% of the examples and 17 inputs, randomly selected from 3 (older version of this dataset with less inputs).

The smallest datasets are provided to test more computationally demanding machine learning algorithms (e.g., SVM).

The classification goal is to predict if the client will subscribe (`yes/no`) a term deposit (variable `y`).

**Attribute Information**


**Input variables: bank client data:**

|No|Title|Description|Data Type|Values|
|---|---|---|---|---|
|1|`age`||numeric||
|2|`job`|type of job|categorical|'admin.', 'blue-collar', 'entrepreneur', 'housemaid', 'management', 'retired', 'self-employed', 'services', 'student', 'technician', 'unemployed', 'unknown'|
|3|`marital`| marital status |categorical| 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed|
|4|`education`| |categorical| 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown'|
|5|`default`| has credit in default? |categorical| 'no','yes','unknown'|
|6|`housing`| has housing loan? |categorical| 'no','yes','unknown'|
|7|`loan`| has personal loan? |categorical| 'no','yes','unknown'|


**Input variables: related with the last contact of the current campaign:**

|No|Title|Description|Data Type|Values|
|---|---|---|---|---|
|8| contact| contact communication type |categorical| 'cellular','telephone'|
|9 | month| last contact month of year |categorical| 'jan', 'feb', 'mar', ..., 'nov', 'dec'|
|10 | day_of_week| last contact day of the week |categorical|'mon','tue','wed','thu','fri'|
|11 | duration| last contact duration, in seconds |numeric||. 

`duration` - **_Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this 
input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model._**

**Input variables: other attributes:**

|No|Title|Description|Data Type|Values|
|---|---|---|---|---|
|12 | `campaign`| number of contacts performed during this campaign and for this client |numeric| includes last contact||
|13 | `pdays`|number of days that passed by after the client was last contacted from a previous campaign |numeric| 999 mean client was not previously contacted||
|14 | `previous`| number of contacts performed before this campaign and for this client |numeric||
|15 | `poutcome`| outcome of the previous marketing campaign |categorical| 'failure','nonexistent','success'|

**Input variables: social and economic context attributes**

|No|Title|Description|Data Type|Values|
|---|---|---|---|---|
|16 | `emp.var.rate`| employment variation rate - quarterly indicator |numeric||
|17 | `cons.price.idx`| consumer price index - monthly indicator |numeric||
|18 | `cons.conf.idx`| consumer confidence index - monthly indicator ||numeric||
|19 | `euribor3m`| euribor 3 month rate - daily indicator |numeric||
|20 | `nr.employed`|number of employees - quarterly indicator |numeric||

**Output variable (desired target):**

|No|Title|Description|Data Type|Values|
|---|---|---|---|---|
|21| `y` | has the client subscribed a term deposit? |binary| 'yes','no'|

Source: https://archive.ics.uci.edu/ml/datasets/bank+marketing


---

## Data load and preview

Для початку завантажимо дані у змінну `data`:

In [ ]:
data <- read.csv("https://raw.githubusercontent.com/kleban/r-course-eng/main/data/banking.csv", 
                 na.strings = c("", " ", "NA", "NULL"), # fix missing as NA if present
                 stringsAsFactors = TRUE) # set strings as factor, we need this for some algorithms
#use + unknown with na.strings if you want to play with missing
#data <- read.csv("data/banking.csv", na.strings = c("", " ", "NA", "NULL", "unknown"))

Переглянемо структуру вибірки даних з `str()`:

In [ ]:
str(data)

Переглянемо вигляд перших рядків даних з `head()`:

In [ ]:
head(data)

Описова статистика факторів:

In [ ]:
summary(data)

Перевіримо вибірку на наявність пропусків:

In [ ]:
library(mice)
md.pattern(data) # OK

In [ ]:
anyNA(data)

---

## Data visualization

**Вік клієнта (age):**

In [ ]:
library(ggplot2)

ggplot(data, aes(age)) + 
    geom_histogram(bins = 20, alpha = 0.5, fill = 'blue', color='black')  + 
    theme_bw()

**Робота клієнта (job):**

In [ ]:
ggplot(data, aes(job)) + 
    geom_bar(aes(fill = job)) + 
    theme_bw()

In [ ]:
library(gmodels)
CrossTable(data$job, data$deposit)
# more loyal to deposits are management, retired, student, unemployed ))

**Сімейний статус (marital):**

In [ ]:
ggplot(data, aes(marital)) + 
    geom_bar(aes(fill = marital)) + 
    theme_bw()

In [ ]:
CrossTable(data$marital, data$deposit)
# married are not very loyal to deposits
# but singles is more loyal

**Освіта (education):**

In [ ]:
ggplot(data, aes(education)) + 
    geom_bar(aes(fill = education)) + 
    theme_bw()

In [ ]:
CrossTable(data$education, data$deposit)
# people with tertiary education is more loyal than other groups

**Дефолт (default):**

In [ ]:
ggplot(data, aes(default)) + 
    geom_bar(aes(fill = default)) + 
    theme_bw()

In [ ]:
CrossTable(data$default, data$deposit)
# defaults not very loyal to deposits, but why? ))))))

**Баланс (balance):**

In [ ]:
ggplot(data, aes(balance)) + 
    geom_histogram(bins = 30, alpha = 0.5, fill = 'blue', color='black')  + 
    theme_bw()

# looks like balance data has outliers

**Наявність кредиту на житло (housing):**

In [ ]:
ggplot(data, aes(housing)) + 
    geom_bar(aes(fill = housing)) + 
    theme_bw()

In [ ]:
CrossTable(data$housing, data$deposit)
# people without housing load logicaly more often can do deposits

**Наявність позики (loan):**

In [ ]:
ggplot(data, aes(loan)) + 
    geom_bar(aes(fill = loan)) + 
    theme_bw()

In [ ]:
CrossTable(data$loan, data$deposit)

**# Тип комунікації (contact):**

In [ ]:
ggplot(data, aes(contact)) + 
    geom_bar(aes(fill = contact)) + 
    theme_bw()

In [ ]:
CrossTable(data$contact, data$deposit)
# cellular communication channel looks like the best way to increase deposits count
# people with cellular devices has more money? 

**День місяця (day):**

In [ ]:
ggplot(data, aes(day)) + 
    geom_histogram(bins = 25, alpha = 0.5, fill = 'blue', color='black')  + 
    theme_bw()

**Місяць (month):**

In [ ]:
ggplot(data, aes(month)) + 
    geom_bar(aes(fill = month)) + 
    theme_bw()

In [ ]:
# So, lets replace our month with ordered factor for correct visualization
library(dplyr)
data <- data |>
    mutate(month = factor(month, levels=c("jan","feb","mar",
               "apr","may","jun","jul","aug","sep",
              "oct","nov","dec"),ordered=TRUE))

In [ ]:
ggplot(data, aes(month)) + 
    geom_bar(aes(fill = month)) + 
    theme_bw()

In [ ]:
CrossTable(data$month, data$deposit)

**Тривалість останнього контакту (duration):**

In [ ]:
ggplot(data, aes(duration)) + 
    geom_histogram(bins = 100, alpha = 0.5, fill = 'blue', color='black')  + 
    theme_bw()

**Кількість контактів протягом поточної кампанії (campaign):**

In [ ]:
ggplot(data, aes(campaign)) + 
    geom_histogram(bins = 30, alpha = 0.5, fill = 'blue', color='black')  + 
    theme_bw()

**Кількість днів від попередньої акції (pday):**

In [ ]:
ggplot(data, aes(pdays)) + 
    geom_histogram(bins = 20, alpha = 0.5, fill = 'blue', color='black')  + 
    theme_bw()

**Кількість контактів до початку поточної кампанії (previous):**

In [ ]:
ggplot(data, aes(previous)) + 
    geom_histogram(bins = 50, alpha = 0.5, fill = 'blue', color='black')  + 
    theme_bw()

**Результат попередньої кампанії (poutcome):**

In [ ]:
ggplot(data, aes(poutcome)) + 
    geom_bar(aes(fill = poutcome)) + 
    theme_bw()

In [ ]:
CrossTable(data$poutcome, data$deposit)
# people with previous success status also loyal for new propositions

**Результат укладання або відсутність укладання договору (deposit):**

In [ ]:
ggplot(data, aes(deposit)) + 
    geom_bar(aes(fill = deposit)) + 
    theme_bw()

In [ ]:
CrossTable(data$deposit)

## Data preprocessing

Перетворимо значення `deposit` до `0` і `1`:

In [ ]:
data$deposit <- ifelse(data$deposit == "yes", 1, 0)

Видалимо `duration`, адже цей параметр чітко вказує на факт укладання угоди, такі дані називаються leak:

In [ ]:
data$duration <- NULL

Створимо новий параметр `pdays_flag`, який вказує чи був контакт з клієнтом раніше:

In [ ]:
data$pdays_flag <- ifelse(data$pdays > 0, 1, 0)

In [ ]:
head(data)

Створимо новий параметр `poutcome_success`, який вказує чи була попередня кампанія з цим клієнтом “успішною для банку”:

In [ ]:
data$poutcoume_success <- ifelse(data$poutcome == "success", 1, 0)

---

## Train/test split

Задаємо seed для генератора випадкових чисел

Train  65%, test 35%

In [ ]:
set.seed(111) # today!
library(caret)
index = createDataPartition(data$deposit, p = 0.65, list = FALSE)
train_data = data[index, ]
test_data = data[-index, ]

In [ ]:
CrossTable(train_data$deposit)
CrossTable(test_data$deposit)

---

## Decision trees with `rpart()`

Для побудови дерев рішень у `R` є ряд пакетів та алгоритмів. Розглянемо пакет `rpart`.

In [ ]:
#install.packages("rpart")
library(rpart)
rpart_model <- rpart(deposit ~ ., train_data)

Виведемо опис моделі:

In [ ]:
rpart_model

Дуже детальний опис:

In [ ]:
summary(rpart_model)

Візуалізуємо дерево рішень:

In [ ]:
# install.packages(c("rattle", "RColorBrewer"))

library(rattle)
library(RColorBrewer)
fancyRpartPlot(rpart_model)

# now you can see how model model works

Створимо два дата-фрейм для для запису результатів моделювання на тестовій вибірці. Одразу додамо у набори даних реальні значення результатів маркетингової акції deposit та модельовані значення

Дані тренувальної вибірки будуть використовуватися для визначення оптимальної cutoff лінії, а тестової для порівняння моделей між собою.

In [ ]:
train_results <- data.frame(No = c(1:nrow(train_data)), 
                            deposit = train_data$deposit, 
                            RPartPredicted = predict(rpart_model, train_data))

test_results <- data.frame(No = c(1:nrow(test_data)),
                           deposit = test_data$deposit, 
                           RPartPredicted = predict(rpart_model, test_data))

head(test_results)

Визначимо оптимальну лінію розподілу на 0 і 1 для тренувальної вибірки за допомогою пакету `InformationValue`:

In [ ]:
library(InformationValue)
optCutOff <- optimalCutoff(train_results$deposit, train_results$RPartPredicted)
optCutOff

Побудуємо `ROC`-криву для тестової вибірки:

In [ ]:
plotROC(test_results$deposit, test_results$RPartPredicted)

Сформуємо набір класів `0` і `1` для тестового набору даних:

In [ ]:
test_results$RPartPredicted_Class <- ifelse(test_results$RPartPredicted > optCutOff, 1, 0)

Confusion matrix:

In [ ]:
cm <- caret::confusionMatrix(factor(test_results$deposit), 
                      factor(test_results$RPartPredicted_Class), 
                      positive = "1")
cm

Переглянемо збалансовану точність класифіції:

In [ ]:
BAcc <- cm$byClass[["Balanced Accuracy"]]
BAcc 

---

## Desicion trees with  PartyKit

Побудуємо дерево рішень за допоомгою пакету `partykit`:

In [ ]:
#install.packages("partykit")
library(partykit)
head(train_data)
party_model <- ctree(deposit ~ ., data = train_data)

Виведемо текстовий опис моделі:

In [ ]:
party_model
# Looks like this model is more complex

Візуалізуємо побудоване дерево рішень:

In [ ]:
plot(party_model)

Конвернтуємо `ctree()` до `rpart()`:

In [ ]:
st <- as.simpleparty(party_model)
plot(st)

Додамо прогнозовані показники до раніше створених дата-фрейму для збору результатів:

In [ ]:
train_results$PartyPredicted <- predict(party_model, train_data)
test_results$PartyPredicted <- predict(party_model, test_data)
head(test_results)

Визначимо оптимальну лінію розділення на класи 0 і 1:

In [ ]:
optCutOff <- optimalCutoff(train_results$deposit, train_results$PartyPredicted)
optCutOff

ROC-крива та AUROC:

In [ ]:
plotROC(test_results$deposit, test_results$PartyPredicted)

Розділимо результати прогнозування на класи:

In [ ]:
test_results$PartyPredicted_Class <- ifelse(test_results$PartyPredicted > optCutOff, 1, 0)

Confusion matrix:

In [ ]:
cm <- caret::confusionMatrix(factor(test_results$deposit), 
                             factor(test_results$PartyPredicted_Class), 
                             positive = "1")
cm

Оцінимо збалансовану точність класифікації:

In [ ]:
BAcc # value for previous model

In [ ]:
BAcc1 <- cm$byClass[["Balanced Accuracy"]]
BAcc1

---

## Desision Tree with c50

Скористаємося алгоритмом `C50` для побудови дерева рішень.
Для початку потрібно виіхдний показник перетворити у категоріальний (`factor`):

In [ ]:
# lets make anew temporary dataset for modeling with target output as factor

In [ ]:
train_data_tmp <- train_data %>%
    mutate(deposit = factor(train_data$deposit, levels = c(0,1)))

test_data_tmp <- test_data %>%
    mutate(deposit = factor(test_data$deposit, levels = c(0,1)))

Побудуємо модель:

In [ ]:
library(C50)
c5_model <- C5.0(deposit ~ ., data = train_data_tmp)

Переглянемо модель:

In [ ]:
summary(c5_model)
# its hard to check the nodes

Здійснимо прогноз значень:

In [ ]:
train_results$C5Predicted <- predict(c5_model, train_data_tmp)
test_results$C5Predicted <- predict(c5_model, test_data_tmp)

ROC-крива та AUROC:

In [ ]:
plotROC(as.numeric(test_results$deposit), as.numeric(test_results$C5Predicted))
# you can see that current algorithm is not very good for this data, partykit is much better

Confusion Matrix:

In [ ]:
cm <- caret::confusionMatrix(factor(test_results$deposit), 
                             test_results$C5Predicted, 
                             positive = "1")
cm

Збалансована точність моделі:

In [ ]:
BAcc # rpart
BAcc1 # partykit

In [ ]:
BAcc2 <- cm$byClass[["Balanced Accuracy"]]
BAcc2
# but balanced accuracy is the best. So this model better classify both good and bad events

---

## RandomForest

You can use `random forest` with default or special training parameters. 

In [ ]:
head(train_results)

In [ ]:
head(train_data)

In [ ]:
#install.packages("randomForest")

In [ ]:
table(train_data$deposit)

In [ ]:
library(randomForest)

rf_model <- randomForest(deposit ~ ., 
                         data=train_data, 
                         ntree=200, 
                         mtry=2, 
                         importance=TRUE) #Should importance of predictors be assessed?

`ntree` - Number of trees to grow. This should not be set to too small a number, to ensure that every input row gets predicted at least a few times.

`mtry` - Number of variables randomly sampled as candidates at each split.

In [ ]:
rf_model

Можемо провести аналіз важливості параметрів у залежності від критерію `зменшення точності` або `зменшення джині`:

In [ ]:
varImpPlot(rf_model)

- [x] `MeanDecreaseAccuracy`: gives a rough estimate of the loss in prediction performance when that particular variable is omitted from the training set. Caveat: if two variables are somewhat redundant, then omitting one of them may not lead to massive gains in prediction performance, but would make the second variable more important.

- [x] `MeanDecreaseGini`: GINI is a measure of node impurity. Think of it like this, if you use this feature to split the data, how pure will the nodes be? Highest purity means that each node contains only elements of a single class. Assessing the decrease in GINI when that feature is omitted leads to an understanding of how important that feature is to split the data correctly.

In [ ]:
train_results$RF <- predict(rf_model, train_data)
test_results$RF <- predict(rf_model, test_data)

In [ ]:
optCutOff <- optimalCutoff(train_results$deposit, train_results$RF)
optCutOff

In [ ]:
test_results$RF_Class = ifelse(test_results$RF > optCutOff, 1, 0)

ROC-крива та AUROC:

In [ ]:
plotROC(as.numeric(test_results$deposit), as.numeric(test_results$RF))

In [ ]:
# Balanced accuracy is much better the before!
cm <- caret::confusionMatrix(factor(test_results$deposit), 
                             factor(test_results$RF_Class), 
                             positive = "1")
cm

Balanced accuracy is hte best for now

---

## xgBoost

Our next step is testing gradient boosting with `xgboost` algorithm.

In [ ]:
library(xgboost)

For complex algorithm like `random forest` or `xgboost` model training is the most important stage.

XGBoost only works with numeric vectors. Therefore, you need to convert all other forms of data into numeric vectors.

In [ ]:
train_labels <- train_data$deposit
test_labels <- test_data$deposit

xgb_train_data <- xgb.DMatrix(data = model.matrix(deposit~., data = train_data),
                              label = train_labels)
xgb_test_data <- xgb.DMatrix(data = model.matrix(deposit~., data = test_data),
                              label = test_labels)
xgb_test_data

We will train decision tree model using the following parameters:

- [x] `objective = "binary:logistic"`: we will train a binary classification model ;
- [x] `max.depth = 2`: the trees won’t be deep, because our case is very simple ;
- [x] `nthread = 2`: the number of CPU threads we are going to use;
- [x] `nrounds = 2`: there will be two passes on the data, the second one will enhance the model by further reducing the difference between ground truth and prediction.

In [ ]:
xgb_model <- xgboost(data = xgb_train_data, 
                     label = train_labels, 
                     max.depth = 2, 
                     #eta = 1, 
                     nthread = 2, 
                     nrounds = 2, 
                     objective = "binary:logistic")
xgb_model

In [ ]:
# predict data
train_results$XGB <- predict(xgb_model, xgb_train_data)
test_results$XGB <- predict(xgb_model, xgb_test_data)

head(test_results)

Optimal cutoff:

In [ ]:
optCutOff <- optimalCutoff(train_results$deposit, train_results$XGB)
optCutOff

In [ ]:
# evaluate classification class
test_results$XGB_Class = ifelse(test_results$XGB > optCutOff, 1, 0)

In [ ]:
plotROC(as.numeric(test_results$deposit), as.numeric(test_results$XGB))

In [ ]:
# Balanced accuracy is not better, random forest wins for now!
cm <- caret::confusionMatrix(factor(test_results$deposit), 
                             factor(test_results$XGB_Class), 
                             positive = "1")
cm

---

## lightgbm 

Light gbm is one of most useful package for machine learning. It has one super power: speed of calculations. While you using very big datasets `randomForest` and `xgBoost` work slow, but `lightgbm` works better.

For this algorithm we should convert our data to special matrices too. So, lets install packages for example:

In [ ]:
# ALERT sometimes you need to unistall Matrix in RSTudio and install it again
library(Matrix)                      
library(lightgbm)

Lets use binning technique for data preprocessing 

In [ ]:
library(scorecard)

In [ ]:
vars_list <- train_data %>%
  select(-deposit) %>%
  names()
vars_list

In [ ]:
bin_class <- woebin(train_data, 
                    y = "deposit", 
                    x = vars_list, 
                    positive = 1, # the value in deposit that indicates event                   
                    bin_num_limit = 20)
# bin_class - to check bins

In [ ]:
train_woe <- woebin_ply(train_data, bin_class)
test_woe <- woebin_ply(test_data, bin_class)
head(train_woe)

In [ ]:
vars_list <- train_woe %>%
  select(-deposit) %>%
  names() 
vars_list

In [ ]:
head(test_woe %>% select(vars_list))

In [ ]:
train_sparse = Matrix(as.matrix(train_woe %>% select(vars_list)), sparse=TRUE)
test_sparse = Matrix(as.matrix(test_woe %>% select(vars_list)), sparse=TRUE)

In [ ]:
lgb.train = lgb.Dataset(data = train_sparse, label = train_woe$deposit, free_raw_data = FALSE)
lgb.test = lgb.Dataset(data = test_sparse, label = test_woe$deposit, free_raw_data = FALSE)


In [ ]:
lgb.grid = list(objective = "binary",
                metric = "auc",
                #save_binary = T,
                max_bin = 32,
                num_leaves = 33)

In [ ]:
lgb.train.cv = lgb.train(params = lgb.grid,
                         data = lgb.train,                         
                         nrounds = 15,                         
                         early_stopping_round = 300,
                         #categorical_feature = categoricals.vec,
                         valids = list(test = lgb.test),
                         verbose = 1) 

In [ ]:
# predict data
train_results$LGBM <- predict(lgb.train.cv, train_sparse)
test_results$LGBM <- predict(lgb.train.cv, test_sparse)

head(test_results)

In [ ]:
# Optimal cutoff:

optCutOff <- optimalCutoff(train_results$deposit, train_results$LGBM)
optCutOff

In [ ]:
# evaluate classification class
test_results$LGBM_Class = ifelse(test_results$LGBM > optCutOff, 1, 0)
plotROC(as.numeric(test_results$deposit), as.numeric(test_results$LGBM))

In [ ]:
# Balanced accuracy is not better, random forest wins for now!
cm <- caret::confusionMatrix(factor(test_results$deposit), 
                             factor(test_results$LGBM_Class), 
                             positive = "1")
cm

---